In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from aws_setup import *

In [3]:
name='fast-ai'

In [9]:
instance = get_instance('fast-ai-gpu-machine')

In [10]:
import paramiko

In [16]:
Path.home()

PosixPath('/Users/andrewshaw')

In [ ]:
instance.public_ip_address

In [44]:
def connect_to_instance(instance, keypath=f'{Path.home()}/.ssh/aws-key-fast-ai.pem', username='ubuntu', timeout=10):
    print('Connecting to SSH...')
    
    client = paramiko.SSHClient()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    retries = 20
    while retries > 0:
        try:
            client.connect(instance.public_ip_address, username=username, key_filename=keypath, timeout=timeout)
            print('Connected!')
            break
        except Exception as e:
            print(f'Exception: {e} Retrying...')
            retries = retries - 1
    return client


In [45]:
client = connect_to_instance(instance)

Connecting to SSH...
Connected!


In [46]:
stdin, stdout, stderr = client.exec_command('ls -l', get_pty=True)
for input in []:
    stdin.write('mypassword\n')
stdout_str = stdout.read().decode('ascii')
stderr_str = stderr.read().decode('ascii')
print("run_sync returned: " + stdout_str)

run_sync returned: total 20
drwxrwxr-x 3 ubuntu ubuntu 4096 Nov  1 22:55 data
drwxrwxr-x 2 ubuntu ubuntu 4096 Mar 29 05:55 efs-mount-point
drwxrwxr-x 6 ubuntu ubuntu 4096 Nov  5 00:29 fastai
drwxr-xr-x 2 root   root   4096 Mar 29 05:27 mount_point
drwxrwxr-x 6 ubuntu ubuntu 4096 Nov  2 16:17 src



In [60]:
def run_command(client, cmd, inputs=[]):
    stdin, stdout, stderr = client.exec_command('ls -l', get_pty=True)
    for inp in inputs:
        # example = 'mypassword\n'
        stdin.write(inp)
    stdout_str = stdout.read().decode('ascii')
    stderr_str = stderr.read().decode('ascii')
    
#     print("run_sync returned: " + stdout_str)
    return stdout_str, stderr_str

In [61]:
def upload_file(client, localpath, remotepath):
#     file = f'{Path.home()}/Projects/ML/fastai/fastai_imagenet/testfile.txt'
    ftp_client=client.open_sftp()
    ftp_client.put(localpath, remotepath)
    ftp_client.close()

In [64]:
out, _ = run_command(client, 'lsblk'); 
print("run_sync returned: " + out)

run_sync returned: total 24
drwxrwxr-x 3 ubuntu ubuntu 4096 Nov  1 22:55 data
drwxrwxr-x 2 ubuntu ubuntu 4096 Mar 29 05:55 efs-mount-point
drwxrwxr-x 6 ubuntu ubuntu 4096 Nov  5 00:29 fastai
drwxr-xr-x 2 root   root   4096 Mar 29 05:27 mount_point
drwxrwxr-x 6 ubuntu ubuntu 4096 Nov  2 16:17 src
-rw-rw-r-- 1 ubuntu ubuntu    9 Mar 29 08:08 testfile.txt



In [47]:
client.exec_command('lsblk')

(<paramiko.ChannelFile from <paramiko.Channel 1 (open) window=2097152 -> <paramiko.Transport at 0x16af4da0 (cipher aes128-ctr, 128 bits) (active; 1 open channel(s))>>>,
 <paramiko.ChannelFile from <paramiko.Channel 1 (open) window=2097152 -> <paramiko.Transport at 0x16af4da0 (cipher aes128-ctr, 128 bits) (active; 1 open channel(s))>>>,
 <paramiko.ChannelFile from <paramiko.Channel 1 (open) window=2097152 -> <paramiko.Transport at 0x16af4da0 (cipher aes128-ctr, 128 bits) (active; 1 open channel(s))>>>)